In [1]:
import os
os.environ['PYSPARK_PYTHON'] = '/opt/conda/envs/python2/bin/python2'

from pyspark import SparkConf, SparkContext
sconf = SparkConf()
sconf.setMaster("local[*]")
sc = SparkContext(conf=sconf)

In [2]:
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)

In [3]:
data = sqlCtx.read.json('../notebook_data/20newsgroups.labelled.json.gz')

In [4]:
data.printSchema()

root
 |-- approved: string (nullable = true)
 |-- article-id: string (nullable = true)
 |-- content: string (nullable = true)
 |-- date: string (nullable = true)
 |-- distribution: string (nullable = true)
 |-- followup-to: string (nullable = true)
 |-- from: string (nullable = true)
 |-- in-reply-to: string (nullable = true)
 |-- keywords: string (nullable = true)
 |-- label: string (nullable = true)
 |-- lines: string (nullable = true)
 |-- message-id: string (nullable = true)
 |-- newsgroups: string (nullable = true)
 |-- nntp-posting-host: string (nullable = true)
 |-- organization: string (nullable = true)
 |-- originator: string (nullable = true)
 |-- path: string (nullable = true)
 |-- references: string (nullable = true)
 |-- reply-to: string (nullable = true)
 |-- sender: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- x-newsreader: string (nullable = true)
 |-- xref: string (nullable = true)



In [5]:
data.registerTempTable('newsgroups')
docs = sqlCtx.sql("SELECT label, content FROM newsgroups")
labelsDistinct = sqlCtx.sql("SELECT DISTINCT(label) FROM newsgroups")
print sqlCtx.sql("SELECT COUNT(DISTINCT(label)) as labelCount FROM newsgroups").collect()

[Row(labelCount=20)]


In [6]:
print labelsDistinct.count()
labelList = labelsDistinct.map(lambda r: r.label.encode('utf-8')).collect()
sc.broadcast(labelList)
from IPython.display import display, HTML
th = "<th>Label</th>"
td = ["<tr><td>" + d + "</td></tr>" for d in labelList]
display(HTML("<table><thead><tr>" + "".join(th) + "</tr></thead><tbody>" + "".join(td) + "</tbody></table>"))

20


Label
rec.sport.hockey
sci.electronics
sci.med
rec.autos
comp.sys.mac.hardware
comp.windows.x
rec.sport.baseball
comp.sys.ibm.pc.hardware
misc.forsale
rec.motorcycles


In [7]:
labels = sqlCtx.sql("SELECT label FROM newsgroups")
labelCounts = labels.map(lambda r: (str(r.label), 1)).reduceByKey(lambda v1, v2: v1 + v2).collect()

from IPython.display import display, HTML
th = "<th>ID</th><th>Label</th><th>Messages</th>"
td = ["<tr><td>" + str(labelList.index(l)) +"</td><td>" + l + "</td><td>" + str(m) +"</tr>" for (l,m) in labelCounts]
display(HTML("<table><thead><tr>" + "".join(th) + "</tr></thead><tbody>" + "".join(td) + "</tbody></table>"))

In [8]:
import re

labelledDocs = docs.map(lambda r: (labelList.index(r.label), r.label.encode('utf-8'), re.sub('\s+', ' ', r.content.encode('utf-8')).strip()))
print labelledDocs.take(10)

[(13, 'alt.atheism', 'Archive-name: atheism/resources Alt-atheism-archive-name: resources Last-modified: 11 December 1992 Version: 1.0 Atheist Resources Addresses of Atheist Organizations USA FREEDOM FROM RELIGION FOUNDATION Darwin fish bumper stickers and assorted other atheist paraphernalia are available from the Freedom From Religion Foundation in the US. Write to: FFRF, P.O. Box 750, Madison, WI 53701. Telephone: (608) 256-8900 EVOLUTION DESIGNS Evolution Designs sell the "Darwin fish". It\'s a fish symbol, like the ones Christians stick on their cars, but with feet and the word "Darwin" written inside. The deluxe moulded 3D plastic fish is $4.95 postpaid in the US. Write to: Evolution Designs, 7119 Laurel Canyon #4, North Hollywood, CA 91605. People in the San Francisco Bay area can get Darwin Fish from Lynn Gold -- try mailing <figmo@netcom.com>. For net people who go to Lynn directly, the price is $4.95 per fish. AMERICAN ATHEIST PRESS AAP publish various atheist books -- critiq

In [19]:
from pyspark.mllib.feature import HashingTF
import nltk
nltk.download()
htf = HashingTF()
htfDocs = labelledDocs.map(lambda (i,l,c) : (i, l, htf.transform(nltk.word_tokenize(c))))
print htfDocs.take(1)

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> x

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 27.0 failed 1 times, most recent failure: Lost task 0.0 in stage 27.0 (TID 425, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/pyspark/rdd.py", line 1295, in takeUpToNumLeft
    yield next(iterator)
  File "<ipython-input-19-18ae91e853b5>", line 5, in <lambda>
  File "/opt/conda/envs/python2/lib/python2.7/site-packages/nltk/tokenize/__init_

In [13]:
from pyspark.mllib.regression import LabeledPoint
inputData = htfDocs.map(lambda (i,l,v): LabeledPoint(i,v))
print inputData.first()
training = inputData
testing = inputData
#training, testing, validation = inputData.randomSplit([0.7,0.2,0.1])
print training.first()
print testing.first()

(13.0,(1048576,[8148,19142,30136,60275,71269,82263,93257,104251,115245,134386,145380,156374,167368,178362,189356,208501,219495,230489,241483,252477,263471,274465,282612,293606,304600,315594,337582,356727,367721,375612,378715,389709,400703,411697,422691,430838,441832,463820,485808,496802,504953,515947,523838,537935,548929,559923,570917,579064,590058,612046,623040,634034,653179,664173,686161,697155,708149,738284,760272,771266,782260,793254,812399,834387,845381,856375,867369,886510,897504,908498,919492,930486,935534,941480,960625,971619,982613,993607,1004601,1034736,1045730],[23.0,33.0,31.0,485.0,269.0,9.0,41.0,26.0,117.0,572.0,890.0,21.0,51.0,3.0,65.0,632.0,164.0,5.0,11.0,1.0,146.0,1660.0,189.0,129.0,4.0,18.0,4.0,76.0,363.0,1.0,3.0,29.0,1.0,53.0,91.0,99.0,661.0,36.0,63.0,1.0,26.0,4.0,1.0,46.0,2.0,40.0,2.0,122.0,56.0,32.0,98.0,18.0,11.0,319.0,40.0,49.0,32.0,199.0,1.0,45.0,24.0,19.0,562.0,52.0,38.0,18.0,25.0,616.0,586.0,58.0,48.0,41.0,1.0,25.0,137.0,108.0,60.0,28.0,32.0,7.0,227.0]))
(13.0,

In [12]:
import timeit
start_time = timeit.default_timer()

from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
model = NaiveBayes.train(training, 1.0)

elapsed = timeit.default_timer() - start_time
print elapsed + " seconds"

328.487263203


In [14]:
predictionAndLabel = testing.map(lambda p : (model.predict(p.features), p.label))
print predictionAndLabel.take(10)
accuracy = 1.0 * predictionAndLabel.filter(lambda (x, v): x == v).count() / testing.count()
print accuracy

[(14.0, 13.0), (14.0, 13.0), (14.0, 13.0), (14.0, 13.0), (14.0, 13.0), (14.0, 13.0), (14.0, 13.0), (14.0, 13.0), (14.0, 13.0), (14.0, 13.0)]
0.0638595789368


### Mess below

In [ ]:
from pyspark.mllib.feature import HashingTF
htf = HashingTF()
hashedTuples = docs.map(lambda r : (labelList.index(r.label), htf.transform(r.content.encode('utf-8').replace('\n', ' ').replace('\r', ''))))
print hashedTuples.first()

In [ ]:
from pyspark.mllib.regression import LabeledPoint
inputData = hashedTuples.map(lambda (c,v): LabeledPoint(c,v))
training, testing, validation = inputData.randomSplit([0.7,0.3,0.0], 5)

In [ ]:
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
model = NaiveBayes.train(training, 1.0)

In [ ]:
predictionAndLabel = testing.map(lambda p : (model.predict(p.features), p.label))
print predictionAndLabel.collect()
accuracy = 1.0 * predictionAndLabel.filter(lambda (x, v): x == v).count() / testing.count()
print accuracy

In [ ]:
from pyspark.mllib.feature import HashingTF
htf = HashingTF()
labels = docs.map(lambda r: (labelList.index(r.label)))
tfDocs = htf.transform(docs.map(lambda r: r.content.encode('utf-8')))
tfDocs.cache()
print tfDocs.first()

In [ ]:
from pyspark.mllib.feature import IDF
hidf = IDF()
hidf = hidf.fit(tfDocs)
tfIdfDocs = hidf.transform(tfDocs)
print tfIdfDocs.first()

featureAndLabelData = labels.zip(tfIdfDocs)

print featureAndLabelData.first()


In [ ]:
from pyspark.mllib.regression import LabeledPoint
inputData = featureAndLabelData.map(lambda (c,v): LabeledPoint(c,v))

In [ ]:
training, testing, validation = inputData.randomSplit([0.9,0.1,0.1], 5)
training.cache()
print training.count()

In [ ]:
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
# Train a naive Bayes model.
model = NaiveBayes.train(training, 1.0)

# Make prediction and test accuracy.
#predictionAndLabel = testing.map(lambda p : (model.predict(p.features), p.label))
#accuracy = 1.0 * predictionAndLabel.filter(lambda (x, v): x == v).count() / testing.count()

In [ ]:
predictionAndLabel = testing.map(lambda p : (model.predict(p.features), p.label))
accuracy = 1.0 * predictionAndLabel.filter(lambda (x, v): x == v).count() / testing.count()
print accuracy